In [1]:
import numpy as np
import pandas as pd
# to display full text
pd.set_option('display.max_colwidth', -1)
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r'graph_eligible_data_v2.csv', encoding = 'utf-8')

In [3]:
df = df.drop('Unnamed: 0' , axis = 1)

In [4]:
df.describe()

,weight,src_user_id,src_favorites_count,src_followers_count,reply_user_id,reply_favorites_count,reply_followers_count,label
count,28235.000000,2.823500e+04,28235.000000,2.823500e+04,2.823500e+04,28235.000000,2.823500e+04,28235.000000
mean,1.631344,2.354490e+08,4465.578785,3.217873e+05,5.832863e+08,4242.330051,1.737657e+04,0.259961
std,2.879345,5.118835e+08,10723.774609,1.182197e+06,7.843626e+08,11571.551719,2.488649e+05,0.438621
min,1.000000,5.800000e+01,0.000000,2.400000e+01,5.800000e+01,0.000000,0.000000e+00,0.000000
25%,1.000000,1.484956e+07,164.000000,6.288000e+03,4.236635e+07,159.000000,1.630000e+02,0.000000
50%,1.000000,2.679228e+07,1277.000000,3.121200e+04,2.370113e+08,842.000000,5.940000e+02,0.000000
75%,1.000000,2.013086e+08,2908.000000,1.078890e+05,7.487968e+08,3482.000000,1.646000e+03,1.000000
max,126.000000,2.725580e+09,90338.000000,1.366748e+07,2.746442e+09,309160.000000,1.406302e+07,1.000000


In [5]:
df.head()

,weight,src_user_id,src_text,src_favorites_count,src_followers_count,src_verified_user,reply_user_id,reply_user_text,reply_favorites_count,reply_followers_count,reply_verified_user,label
0,3,1539100118,Black teenage boys are not men. They are children. Stop referring to a 17 year old as a man. You are killing children. #ferguson,413,59,False,1539100118,Black teenage boys are not men. They are children. Stop referring to a 17 year old as a man. You are killing children. #ferguson @HighPlainsCoot fuck. Off. @AlexisAStevens yeah that tweet was written before that was clarified. Sentiment still holds. Thanks.,439,58,False,0
1,1,1539100118,Black teenage boys are not men. They are children. Stop referring to a 17 year old as a man. You are killing children. #ferguson,413,59,False,587926686,@annaxsweat @NeoSoulPol Same thing when #Trayvon was killed by #Zimmerman the wanna-be cop. 17yrs old #Black teen is a man to them.,42176,2718,False,0
2,2,1539100118,Black teenage boys are not men. They are children. Stop referring to a 17 year old as a man. You are killing children. #ferguson,413,59,False,21461995,@annaxsweat http://t.co/Z7Qm4iaoSR @annaxsweat #StopThugCops We have to take action to stop this insanity. #Ferguson,5472,1437,False,0
3,1,1539100118,Black teenage boys are not men. They are children. Stop referring to a 17 year old as a man. You are killing children. #ferguson,413,59,False,2411474370,"@annaxsweat @KidFriendlyCamb 17 year olds can be prosecuted as adults, regardless of color. Sorry, but try again.",1088,76,False,0
4,3,1539100118,Black teenage boys are not men. They are children. Stop referring to a 17 year old as a man. You are killing children. #ferguson,413,59,False,2437571275,"@annaxsweat well, he damn sure ain't a child @annaxsweat the response of a leftist toker. it's just like stuttering w/ u ppl. @annaxsweat such talk from a lady(?). senseless, so u r blocked",0,57,False,0


In [6]:
df_slice = df[['src_user_id', 'src_text','reply_user_id', 'reply_user_text']]

In [7]:
import re
# function for cleaning data
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt

df['clean_text_initiator'] = np.vectorize(remove_pattern)(df['src_text'], "@[\w]*")
df['clean_text_reply'] = np.vectorize(remove_pattern)(df['reply_user_text'], "@[\w]*")

In [8]:
df['clean_text_initiator'] = df['clean_text_initiator'].str.replace("[^a-zA-Z#]", " ")
df['clean_text_reply'] = df['clean_text_reply'].str.replace("[^a-zA-Z#]", " ")

In [9]:
df['clean_text_initiator'] = df.clean_text_initiator.apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))
df['clean_text_reply'] = df.clean_text_reply.apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))

In [10]:
df.clean_text_initiator = df.clean_text_initiator.apply(lambda x: x.split())
df.clean_text_reply = df.clean_text_reply.apply(lambda x: x.split())

In [11]:
from nltk.stem.porter import * 
stemmer = PorterStemmer() 
tokenized_tweet_text = df.clean_text_initiator.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming
tokenized_tweet_text1 = df.clean_text_reply.apply(lambda x: [stemmer.stem(i) for i in x])

In [12]:
for i in range(len(tokenized_tweet_text)):
    tokenized_tweet_text[i] = ' '.join(tokenized_tweet_text[i])    
df['clean_text_initiator'] = tokenized_tweet_text

for i in range(len(tokenized_tweet_text1)):
    tokenized_tweet_text1[i] = ' '.join(tokenized_tweet_text1[i])    
df['clean_text_reply'] = tokenized_tweet_text1

In [13]:
df.head()

,weight,src_user_id,src_text,src_favorites_count,src_followers_count,src_verified_user,reply_user_id,reply_user_text,reply_favorites_count,reply_followers_count,reply_verified_user,label,clean_text_initiator,clean_text_reply
0,3,1539100118,Black teenage boys are not men. They are children. Stop referring to a 17 year old as a man. You are killing children. #ferguson,413,59,False,1539100118,Black teenage boys are not men. They are children. Stop referring to a 17 year old as a man. You are killing children. #ferguson @HighPlainsCoot fuck. Off. @AlexisAStevens yeah that tweet was written before that was clarified. Sentiment still holds. Thanks.,439,58,False,0,black teenag boy they children stop refer year kill children #ferguson,black teenag boy they children stop refer year kill children #ferguson fuck yeah that tweet written befor that clarifi sentiment still hold thank
1,1,1539100118,Black teenage boys are not men. They are children. Stop referring to a 17 year old as a man. You are killing children. #ferguson,413,59,False,587926686,@annaxsweat @NeoSoulPol Same thing when #Trayvon was killed by #Zimmerman the wanna-be cop. 17yrs old #Black teen is a man to them.,42176,2718,False,0,black teenag boy they children stop refer year kill children #ferguson,same thing when #trayvon kill #zimmerman wanna #black teen them
2,2,1539100118,Black teenage boys are not men. They are children. Stop referring to a 17 year old as a man. You are killing children. #ferguson,413,59,False,21461995,@annaxsweat http://t.co/Z7Qm4iaoSR @annaxsweat #StopThugCops We have to take action to stop this insanity. #Ferguson,5472,1437,False,0,black teenag boy they children stop refer year kill children #ferguson,http iaosr #stopthugcop have take action stop thi insan #ferguson
3,1,1539100118,Black teenage boys are not men. They are children. Stop referring to a 17 year old as a man. You are killing children. #ferguson,413,59,False,2411474370,"@annaxsweat @KidFriendlyCamb 17 year olds can be prosecuted as adults, regardless of color. Sorry, but try again.",1088,76,False,0,black teenag boy they children stop refer year kill children #ferguson,year old prosecut adult regardless color sorri again
4,3,1539100118,Black teenage boys are not men. They are children. Stop referring to a 17 year old as a man. You are killing children. #ferguson,413,59,False,2437571275,"@annaxsweat well, he damn sure ain't a child @annaxsweat the response of a leftist toker. it's just like stuttering w/ u ppl. @annaxsweat such talk from a lady(?). senseless, so u r blocked",0,57,False,0,black teenag boy they children stop refer year kill children #ferguson,well damn sure child respons leftist toker just like stutter such talk from ladi senseless block


In [14]:
df['src_verified_user'] = df['src_verified_user'].apply(lambda x: 1 if "True" else 0)

In [15]:
df['reply_verified_user'] = df['reply_verified_user'].apply(lambda x: 1 if "True" else 0)

In [16]:
 df['reply_label'] = 0

In [17]:
df.shape

(28235, 15)

# Antonym and jaccard score

In [18]:
from nltk.tokenize import word_tokenize

In [19]:
from nltk.corpus import wordnet

In [20]:
def jaccard_similarity_score(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [21]:
#threshold = 0.05
c = 1
for i in range (0, len(df)):
    print(i)
    initiator_label = df.iloc[i]['label']
    antonyms = []
    initiator_token = []
    reply_token = []
    antonyms_count = 0
    line_initiator = df.iloc[i]['clean_text_initiator']
    line_reply = df.iloc[i]['clean_text_reply']
    
    #tokens for initiators sentences
    #for l in line_initiator:
    initiator_token = word_tokenize(line_initiator)
    if (c == 1):
        print(initiator_token)
        c = c + 1
    
    #tokens for reply sentences
    #for l1 in line_reply:
    reply_token = word_tokenize(line_reply)
    
    #generate antonyms for initiator sentence
    for token in initiator_token:
        for syn in wordnet.synsets(token):
            for l2 in syn.lemmas():
                if l2.antonyms():
                    antonyms.append(l2.antonyms()[0].name())
    
    #comparing antonyms of initiator sentence with reply sentence
    for word in antonyms:
        for compared_word in reply_token:
            if (word == compared_word):
                print("word ",word, " compared ", compared_word)
                antonyms_count = antonyms_count + 1
            
    #score calculation
    #antonym_score = (antonyms_count) / (len(initiator_token) + len(reply_token))
    #print(antonym_score)
    
    jaccard_score = jaccard_similarity_score(df.iloc[i]['clean_text_initiator'], df.iloc[i]['clean_text_reply'])
    
    count = 0
    #assigning labels to reply nodes
    if (antonyms_count >= 1 or jaccard_score < 0.5):
        print(count)
        count = count + 1
        df.loc[i,'reply_label'] = 1 - initiator_label
        #print(" reply label for opposites: ", df.loc[i,'reply_label'])
        #print("if:", df.iloc[i]['label'], " reply: ", )
    else:
        df.loc[i,'reply_label'] = initiator_label
    

0
['black', 'teenag', 'boy', 'they', 'children', 'stop', 'refer', 'year', 'kill', 'children', '#', 'ferguson']
0
1
0
2
0
3
0
4
0
5
0
6
0
7
0
8
0
9
0
10
0
11
12
0
13
14
0
15
16
0
17
0
18
0
19
20
0
21
0
22
0
23
0
24
0
25
26
27
0
28
29
30
0
31
0
32
0
33
0
34
0
35
36
37
38
39
0
40
0
41
0
42
43
44
45
0
46
47
0
48
49
50
0
51
0
52
0
53
0
54
0
55
56
0
57
0
58
0
59
0
60
0
61
0
62
63
0
64
0
65
0
66
67
0
68
0
69
0
70
0
71
0
72
0
73
74
0
75
0
76
0
77
0
78
0
79
0
80
0
81
82
83
84
0
85
0
86
0
87
0
88
89
0
90
0
91
92
0
93
0
94
0
95
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
0
96
0
97
0
98
0
99
0
100
0
101
0
102
0
103
0
104
0
105
0
106
0
107
0
108
0
109
0
110
111
0
112
0
113
0
114
0
115
0
116
117
0
118
0
119
0
120
121
122
0
123
124
125
126
127
0
128
129
0
130
131
0
132
0
133
0
134
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
0
135
0
136
0
137
word  white  comp

374
0
375
0
376
0
377
0
378
379
0
380
381
382
383
384
385
386
0
387
388
389
390
0
391
392
393
0
394
0
395
396
0
397
0
398
0
399
0
400
0
401
0
402
403
0
404
0
405
406
0
407
0
408
0
409
0
410
0
411
0
412
0
413
414
0
415
0
416
0
417
0
418
word  lack  compared  lack
0
419
0
420
0
421
0
422
0
423
0
424
0
425
0
426
427
0
428
0
429
0
430
0
431
0
432
0
433
0
434
0
435
0
436
0
437
0
438
439
0
440
0
441
442
0
443
0
444
445
0
446
0
447
0
448
0
449
0
450
0
451
0
452
0
453
0
454
0
455
0
456
0
457
word  keep  compared  keep
0
458
0
459
0
460
0
461
0
462
0
463
0
464
0
465
466
0
467
0
468
0
469
0
470
0
471
0
472
0
473
0
474
0
475
0
476
0
477
478
479
480
0
481
0
482
0
483
0
484
0
485
0
486
0
487
0
488
0
489
490
0
491
492
0
493
0
494
0
495
496
0
497
0
498
0
499
0
500
0
501
0
502
0
503
504
0
505
0
506
0
507
508
0
509
510
0
511
512
513
0
514
0
515
0
516
0
517
518
0
519
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  black  compared 

0
1183
0
1184
0
1185
0
1186
0
1187
1188
word  black  compared  black
word  black  compared  black
word  black  compared  black
word  black  compared  black
0
1189
0
1190
0
1191
0
1192
word  black  compared  black
word  black  compared  black
word  black  compared  black
word  black  compared  black
0
1193
0
1194
0
1195
0
1196
word  black  compared  black
word  black  compared  black
word  black  compared  black
word  black  compared  black
word  black  compared  black
word  black  compared  black
word  black  compared  black
word  black  compared  black
word  black  compared  black
word  black  compared  black
word  black  compared  black
word  black  compared  black
word  black  compared  black
word  black  compared  black
word  black  compared  black
word  black  compared  black
0
1197
0
1198
0
1199
0
1200
0
1201
0
1202
1203
0
1204
0
1205
0
1206
0
1207
0
1208
0
1209
0
1210
word  black  compared  black
word  black  compared  black
word  black  compared  black
word  black  compared  bl

1690
0
1691
0
1692
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
0
1693
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
0
1694
0
1695
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
0
1696
0
1697
0
1698
word  white

1724
1725
0
1726
0
1727
0
1728
0
1729
0
1730
1731
0
1732
0
1733
0
1734
0
1735
1736
0
1737
0
1738
0
1739
1740
0
1741
0
1742
0
1743
1744
0
1745
1746
0
1747
1748
1749
0
1750
0
1751
0
1752
0
1753
0
1754
0
1755
0
1756
0
1757
0
1758
0
1759
0
1760
0
1761
0
1762
1763
0
1764
1765
1766
1767
0
1768
0
1769
0
1770
1771
0
1772
0
1773
0
1774
1775
1776
1777
0
1778
1779
0
1780
0
1781
0
1782
0
1783
0
1784
0
1785
1786
0
1787
0
1788
0
1789
0
1790
0
1791
1792
1793
1794
1795
0
1796
0
1797
0
1798
0
1799
0
1800
0
1801
0
1802
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  black  compared  black
word  black  compared  black
word  black  compared  black
word  black  compared  black
0
1803
0
1804
word  black  compared  black
word  black  compared  black
word  black  compared  black
word  black  compared  black
0
1805
word  black  compared  black
word  black  compared  black
word  black  compared  black
word  black  compared  black
0
1806
0

...

23681
0
23682
23683
23684
0
23685
23686
0
23687
23688
23689
23690
23691
23692
0
23693
23694
23695
0
23696
23697
0
23698
23699
0
23700
23701
0
23702
23703
0
23704
23705
23706
0
23707
0
23708
23709
0
23710
0
23711
0
23712
0
23713
0
23714
23715
0
23716
0
23717
23718
23719
0
23720
0
23721
0
23722
23723
word  come  compared  come
0
23724
word  come  compared  come
0
23725
0
23726
0
23727
0
23728
0
23729
0
23730
23731
0
23732
0
23733
23734
0
23735
23736
0
23737
0
23738
23739
23740
23741
0
23742
23743
0
23744
0
23745
0
23746
0
23747
0
23748
0
23749
0
23750
0
23751
0
23752
23753
0
23754
23755
23756
0
23757
0
23758
0
23759
23760
0
23761
0
23762
23763
0
23764
0
23765
0
23766
0
23767
0
23768
0
23769
23770
0
23771
0
23772
0
23773
0
23774
0
23775
0
23776
0
23777
0
23778
0
23779
0
23780
0
23781
0
23782
0
23783
0
23784
0
23785
0
23786
0
23787
23788
0
23789
word  stay  compared  stay
word  day  compared  day
0
23790
0
23791
0
23792
0
23793
0
23794
0
23795
word  stay  compared  stay
word  day  compared

0
24568
0
24569
0
24570
0
24571
0
24572
0
24573
0
24574
0
24575
0
24576
24577
24578
0
24579
0
24580
0
24581
0
24582
0
24583
0
24584
0
24585
0
24586
0
24587
0
24588
0
24589
0
24590
0
24591
0
24592
0
24593
24594
24595
24596
24597
24598
24599
24600
0
24601
0
24602
24603
0
24604
0
24605
0
24606
24607
24608
0
24609
0
24610
0
24611
0
24612
24613
0
24614
24615
24616
24617
24618
24619
24620
24621
24622
24623
24624
24625
24626
24627
24628
24629
24630
24631
0
24632
0
24633
0
24634
24635
0
24636
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
0
24637
24638
24639
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
0
24640
0
24641
0
24642
0
24643
0
24644
0
24645
0
24646
0
24647
0
24648
word  white  compared  white
word  white  compared  white
word  white  compared  white
word  white  compared  white
0
24649
0
24650
24651
0
24652
0
24653
0
24654
0
24655
0
24656
0
24657


0
25370
25371
0
25372
0
25373
0
25374
25375
0
25376
word  make  compared  make
0
25377
0
25378
0
25379
0
25380
0
25381
0
25382
0
25383
0
25384
0
25385
25386
0
25387
25388
0
25389
0
25390
0
25391
0
25392
0
25393
25394
0
25395
0
25396
0
25397
0
25398
0
25399
0
25400
word  differ  compared  differ
0
25401
0
25402
0
25403
0
25404
0
25405
25406
0
25407
0
25408
0
25409
0
25410
0
25411
25412
0
25413
0
25414
0
25415
0
25416
25417
0
25418
0
25419
25420
0
25421
0
25422
0
25423
0
25424
0
25425
0
25426
0
25427
0
25428
0
25429
0
25430
0
25431
0
25432
0
25433
0
25434
0
25435
0
25436
0
25437
0
25438
0
25439
25440
0
25441
0
25442
0
25443
0
25444
0
25445
0
25446
0
25447
0
25448
25449
word  go  compared  go
0
25450
25451
0
25452
25453
25454
0
25455
25456
0
25457
25458
25459
25460
25461
0
25462
25463
0
25464
0
25465
0
25466
0
25467
0
25468
0
25469
0
25470
0
25471
0
25472
0
25473
0
25474
0
25475
0
25476
0
25477
0
25478
0
25479
0
25480
0
25481
0
25482
0
25483
0
25484
0
25485
0
25486
0
25487
0
25488
0
25489

0
26373
0
26374
0
26375
0
26376
26377
0
26378
26379
0
26380
0
26381
0
26382
0
26383
0
26384
0
26385
26386
0
26387
0
26388
0
26389
0
26390
0
26391
0
26392
0
26393
0
26394
0
26395
26396
0
26397
0
26398
word  show  compared  show
0
26399
0
26400
0
26401
0
26402
0
26403
0
26404
0
26405
0
26406
0
26407
0
26408
26409
0
26410
0
26411
0
26412
0
26413
26414
0
26415
0
26416
0
26417
0
26418
0
26419
26420
0
26421
0
26422
0
26423
0
26424
0
26425
0
26426
0
26427
0
26428
0
26429
0
26430
0
26431
0
26432
0
26433
0
26434
0
26435
0
26436
0
26437
0
26438
0
26439
26440
0
26441
0
26442
0
26443
0
26444
0
26445
0
26446
0
26447
0
26448
0
26449
0
26450
0
26451
0
26452
0
26453
0
26454
0
26455
0
26456
26457
0
26458
0
26459
0
26460
0
26461
0
26462
0
26463
0
26464
0
26465
0
26466
0
26467
0
26468
26469
0
26470
0
26471
0
26472
0
26473
0
26474
0
26475
0
26476
0
26477
0
26478
0
26479
0
26480
0
26481
word  night  compared  night
0
26482
0
26483
0
26484
0
26485
0
26486
0
26487
26488
0
26489
0
26490
0
26491
0
26492
0
2649

27331
0
27332
0
27333
0
27334
0
27335
0
27336
0
27337
27338
0
27339
0
27340
0
27341
0
27342
0
27343
0
27344
27345
0
27346
0
27347
27348
0
27349
0
27350
0
27351
0
27352
0
27353
27354
27355
0
27356
0
27357
27358
0
27359
0
27360
27361
0
27362
0
27363
0
27364
0
27365
0
27366
0
27367
27368
0
27369
0
27370
0
27371
0
27372
word  start  compared  start
word  start  compared  start
0
27373
0
27374
27375
0
27376
0
27377
0
27378
0
27379
0
27380
0
27381
0
27382
0
27383
0
27384
0
27385
0
27386
0
27387
0
27388
0
27389
0
27390
0
27391
27392
0
27393
0
27394
27395
0
27396
0
27397
27398
0
27399
0
27400
27401
0
27402
27403
0
27404
27405
27406
27407
0
27408
27409
0
27410
0
27411
27412
0
27413
0
27414
0
27415
0
27416
0
27417
0
27418
0
27419
0
27420
0
27421
0
27422
0
27423
0
27424
0
27425
0
27426
27427
0
27428
0
27429
0
27430
0
27431
0
27432
0
27433
0
27434
27435
0
27436
0
27437
0
27438
0
27439
0
27440
0
27441
0
27442
0
27443
0
27444
0
27445
0
27446
0
27447
0
27448
0
27449
27450
0
27451
0
27452
27453
0
2745

In [22]:
df['reply_label'].describe()

count    28235.000000
mean     0.642146    
std      0.479377    
min      0.000000    
25%      0.000000    
50%      1.000000    
75%      1.000000    
max      1.000000    
Name: reply_label, dtype: float64

In [23]:
count_0 = 0
count_1 = 0
for i in range(0, len(df)):
    if (df.iloc[i]['reply_label'] == 0):
        count_0 = count_0 + 1
    elif (df.iloc[i]['reply_label'] == 1):
        count_1 = count_1 + 1
print("0: ", count_0)
print("1: ", count_1)
percent_0 = (count_0) / (len(df) ) * 100
print(percent_0)

percent_1 = (count_1) / (len(df) ) * 100
print(percent_1)

0:  10104
1:  18131
35.78537276429963
64.21462723570038


In [24]:
sum(df['reply_label'].apply(lambda x: x if 1 else 0))

18131

In [25]:
df.head()

,weight,src_user_id,src_text,src_favorites_count,src_followers_count,src_verified_user,reply_user_id,reply_user_text,reply_favorites_count,reply_followers_count,reply_verified_user,label,clean_text_initiator,clean_text_reply,reply_label
0,3,1539100118,Black teenage boys are not men. They are children. Stop referring to a 17 year old as a man. You are killing children. #ferguson,413,59,1,1539100118,Black teenage boys are not men. They are children. Stop referring to a 17 year old as a man. You are killing children. #ferguson @HighPlainsCoot fuck. Off. @AlexisAStevens yeah that tweet was written before that was clarified. Sentiment still holds. Thanks.,439,58,1,0,black teenag boy they children stop refer year kill children #ferguson,black teenag boy they children stop refer year kill children #ferguson fuck yeah that tweet written befor that clarifi sentiment still hold thank,1
1,1,1539100118,Black teenage boys are not men. They are children. Stop referring to a 17 year old as a man. You are killing children. #ferguson,413,59,1,587926686,@annaxsweat @NeoSoulPol Same thing when #Trayvon was killed by #Zimmerman the wanna-be cop. 17yrs old #Black teen is a man to them.,42176,2718,1,0,black teenag boy they children stop refer year kill children #ferguson,same thing when #trayvon kill #zimmerman wanna #black teen them,1
2,2,1539100118,Black teenage boys are not men. They are children. Stop referring to a 17 year old as a man. You are killing children. #ferguson,413,59,1,21461995,@annaxsweat http://t.co/Z7Qm4iaoSR @annaxsweat #StopThugCops We have to take action to stop this insanity. #Ferguson,5472,1437,1,0,black teenag boy they children stop refer year kill children #ferguson,http iaosr #stopthugcop have take action stop thi insan #ferguson,1
3,1,1539100118,Black teenage boys are not men. They are children. Stop referring to a 17 year old as a man. You are killing children. #ferguson,413,59,1,2411474370,"@annaxsweat @KidFriendlyCamb 17 year olds can be prosecuted as adults, regardless of color. Sorry, but try again.",1088,76,1,0,black teenag boy they children stop refer year kill children #ferguson,year old prosecut adult regardless color sorri again,1
4,3,1539100118,Black teenage boys are not men. They are children. Stop referring to a 17 year old as a man. You are killing children. #ferguson,413,59,1,2437571275,"@annaxsweat well, he damn sure ain't a child @annaxsweat the response of a leftist toker. it's just like stuttering w/ u ppl. @annaxsweat such talk from a lady(?). senseless, so u r blocked",0,57,1,0,black teenag boy they children stop refer year kill children #ferguson,well damn sure child respons leftist toker just like stutter such talk from ladi senseless block,1


# not to be executed

In [26]:
df.to_csv('charlie_replynode_labelling.csv')

In [3]:
df = pd.read_csv('charlie_replynode_labelling.csv')

In [27]:
df.nunique()

weight                   45   
src_user_id              496  
src_text                 1010 
src_favorites_count      549  
src_followers_count      958  
src_verified_user        1    
reply_user_id            10738
reply_user_text          14068
reply_favorites_count    5709 
reply_followers_count    4315 
reply_verified_user      1    
label                    2    
clean_text_initiator     1008 
clean_text_reply         13292
reply_label              2    
dtype: int64

In [28]:
len(df[df['src_user_id'] == df['reply_user_id']])

1585

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28235 entries, 0 to 28234
Data columns (total 15 columns):
weight                   28235 non-null int64
src_user_id              28235 non-null int64
src_text                 28235 non-null object
src_favorites_count      28235 non-null int64
src_followers_count      28235 non-null int64
src_verified_user        28235 non-null int64
reply_user_id            28235 non-null int64
reply_user_text          28235 non-null object
reply_favorites_count    28235 non-null int64
reply_followers_count    28235 non-null int64
reply_verified_user      28235 non-null int64
label                    28235 non-null int64
clean_text_initiator     28235 non-null object
clean_text_reply         28235 non-null object
reply_label              28235 non-null int64
dtypes: int64(11), object(4)
memory usage: 3.2+ MB


In [29]:
df = df.drop('Unnamed: 0' , axis = 1)

KeyError: "['Unnamed: 0'] not found in axis"

In [136]:
#18992 rows

In [30]:
df.shape[0]

28235

In [31]:
indices = []
for i in range(0, df.shape[0]):
    indices.append(i)
len(indices)

28235

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28235 entries, 0 to 28234
Data columns (total 15 columns):
weight                   28235 non-null int64
src_user_id              28235 non-null int64
src_text                 28235 non-null object
src_favorites_count      28235 non-null int64
src_followers_count      28235 non-null int64
src_verified_user        28235 non-null int64
reply_user_id            28235 non-null int64
reply_user_text          28235 non-null object
reply_favorites_count    28235 non-null int64
reply_followers_count    28235 non-null int64
reply_verified_user      28235 non-null int64
label                    28235 non-null int64
clean_text_initiator     28235 non-null object
clean_text_reply         28235 non-null object
reply_label              28235 non-null int64
dtypes: int64(11), object(4)
memory usage: 3.2+ MB


In [33]:
df.columns

Index(['weight', 'src_user_id', 'src_text', 'src_favorites_count',
       'src_followers_count', 'src_verified_user', 'reply_user_id',
       'reply_user_text', 'reply_favorites_count', 'reply_followers_count',
       'reply_verified_user', 'label', 'clean_text_initiator',
       'clean_text_reply', 'reply_label'],
      dtype='object')

In [34]:
columns = ['user_id', 'text', 'favorites_count', 'verified', 'followers_count', 'label']

In [35]:
df_graph = pd.DataFrame(data = np.zeros((df.shape[0], 6)), index = indices, columns = columns)

In [36]:
list_of_initiators = df['src_user_id'].unique()

In [37]:
#adding data in linear fashion asll user ids in one column for src_ids here
for i in range (0, len(list_of_initiators)):
    print(i)
    count = 0
    for j in range(0, len(df)):
        if ((list_of_initiators[i] == df.iloc[j]['src_user_id']) & (count == 0)):
            count = count + 1
            df_graph.loc[i,'user_id'] = df.iloc[j]['src_user_id']
            df_graph.loc[i,'text'] = df.iloc[j]['clean_text_initiator']
            df_graph.loc[i,'favorites_count'] = df.iloc[j]['src_favorites_count']
            df_graph.loc[i,'followers_count'] = df.iloc[j]['src_followers_count']
            df_graph.loc[i,'verified'] = df.iloc[j]['src_verified_user']
            df_graph.loc[i,'label'] = df.iloc[j]['label']


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [38]:
df_graph.iloc[1005]

user_id            0
text               0
favorites_count    0
verified           0
followers_count    0
label              0
Name: 1005, dtype: object

In [39]:
list_of_replies = df['reply_user_id'].unique()

# not to be executed

In [40]:
df_graph.to_csv('graph-data.csv')

In [6]:
df_graph = pd.read_csv('graph-data.csv')

In [7]:
df_graph = df_graph.drop('Unnamed: 0' , axis = 1)

In [41]:
df_graph['user_id'].nunique()

497

In [42]:
df_graph.iloc[1004]

user_id            0
text               0
favorites_count    0
verified           0
followers_count    0
label              0
Name: 1004, dtype: object

In [43]:
type(df.iloc[0]['clean_text_reply'])

str

In [44]:
len(list_of_replies)

10738

In [45]:
#adding data in linear fashion as all user ids in one column for reply ids here
k = len(list_of_initiators)
for i in range (0, len(list_of_replies)):
    k = k + 1
    print(i)
    count = 0
    text = ''
    for j in range(0, len(df)):
        if (list_of_replies[i] == df.iloc[j]['reply_user_id']):
            if (count == 0):
                df_graph.loc[k,'user_id'] = df.iloc[j]['reply_user_id']
                df_graph.loc[k,'favorites_count'] = df.iloc[j]['reply_favorites_count']
                df_graph.loc[k,'followers_count'] = df.iloc[j]['reply_followers_count']
                df_graph.loc[k,'verified'] = df.iloc[j]['reply_verified_user']
                df_graph.loc[k,'label'] = df.iloc[j]['reply_label']
                count = count + 1
                    
            text = text + str(df.iloc[j]['clean_text_reply'])

    df_graph.loc[k,'text'] = text

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699


5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338


6778
6779
6780
6781
6782
6783
6784
6785
6786
6787
6788
6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977


8417
8418
8419
8420
8421
8422
8423
8424
8425
8426
8427
8428
8429
8430
8431
8432
8433
8434
8435
8436
8437
8438
8439
8440
8441
8442
8443
8444
8445
8446
8447
8448
8449
8450
8451
8452
8453
8454
8455
8456
8457
8458
8459
8460
8461
8462
8463
8464
8465
8466
8467
8468
8469
8470
8471
8472
8473
8474
8475
8476
8477
8478
8479
8480
8481
8482
8483
8484
8485
8486
8487
8488
8489
8490
8491
8492
8493
8494
8495
8496
8497
8498
8499
8500
8501
8502
8503
8504
8505
8506
8507
8508
8509
8510
8511
8512
8513
8514
8515
8516
8517
8518
8519
8520
8521
8522
8523
8524
8525
8526
8527
8528
8529
8530
8531
8532
8533
8534
8535
8536
8537
8538
8539
8540
8541
8542
8543
8544
8545
8546
8547
8548
8549
8550
8551
8552
8553
8554
8555
8556
8557
8558
8559
8560
8561
8562
8563
8564
8565
8566
8567
8568
8569
8570
8571
8572
8573
8574
8575
8576
8577
8578
8579
8580
8581
8582
8583
8584
8585
8586
8587
8588
8589
8590
8591
8592
8593
8594
8595
8596
8597
8598
8599
8600
8601
8602
8603
8604
8605
8606
8607
8608
8609
8610
8611
8612
8613
8614
8615
8616


10047
10048
10049
10050
10051
10052
10053
10054
10055
10056
10057
10058
10059
10060
10061
10062
10063
10064
10065
10066
10067
10068
10069
10070
10071
10072
10073
10074
10075
10076
10077
10078
10079
10080
10081
10082
10083
10084
10085
10086
10087
10088
10089
10090
10091
10092
10093
10094
10095
10096
10097
10098
10099
10100
10101
10102
10103
10104
10105
10106
10107
10108
10109
10110
10111
10112
10113
10114
10115
10116
10117
10118
10119
10120
10121
10122
10123
10124
10125
10126
10127
10128
10129
10130
10131
10132
10133
10134
10135
10136
10137
10138
10139
10140
10141
10142
10143
10144
10145
10146
10147
10148
10149
10150
10151
10152
10153
10154
10155
10156
10157
10158
10159
10160
10161
10162
10163
10164
10165
10166
10167
10168
10169
10170
10171
10172
10173
10174
10175
10176
10177
10178
10179
10180
10181
10182
10183
10184
10185
10186
10187
10188
10189
10190
10191
10192
10193
10194
10195
10196
10197
10198
10199
10200
10201
10202
10203
10204
10205
10206
10207
10208
10209
10210
10211
10212
1021

In [32]:
#adding data in linear fashion asll user ids in one column for reply ids here
k = 17991
for i in range (0, 1006):
    k = k + 1
    print(i)
    count = 0
    text = ''
    for j in range(0, len(df)):
        if (list_of_replies[i] == df.iloc[j]['reply_user_id']):
            if (count == 0):
                df_graph.loc[k,'user_id'] = df.iloc[j]['reply_user_id']
                df_graph.loc[k,'favorites_count'] = df.iloc[j]['reply_favorites_count']
                df_graph.loc[k,'followers_count'] = df.iloc[j]['reply_followers_count']
                df_graph.loc[k,'verified'] = df.iloc[j]['reply_verified_user']
                df_graph.loc[k,'label'] = df.iloc[j]['reply_label']
                count = count + 1
                    
            text = text + str(df.iloc[j]['clean_text_reply'])

    df_graph.loc[k,'text'] = text

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [46]:
df_graph.shape

(28235, 6)

In [47]:
df_graph.describe()

,user_id,favorites_count,verified,followers_count,label
count,2.823500e+04,28235.000000,28235.000000,2.823500e+04,28235.000000
mean,2.307857e+08,1503.225607,0.397875,8.349708e+03,0.257482
std,5.715640e+08,6654.006094,0.489468,1.809650e+05,0.437255
min,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000
25%,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000
50%,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000
75%,1.045726e+08,341.000000,1.000000,3.010000e+02,1.000000
max,2.746442e+09,306610.000000,1.000000,1.405974e+07,1.000000


df_graph.to_csv('graph-datav2.csv')

df_graph.to_csv('graph-datav3.csv')

In [48]:
#converting labels to user suspicious score (for src_ids)
for i in range (0, len(list_of_initiators)):
    print(i)
    label_0 = 0
    label_1 = 0
    for j in range(0, len(df)):
            if (list_of_initiators[i] == df.iloc[j]['src_user_id']):
                if (df.iloc[j]['label'] == 0):
                    label_0 = label_0 + 1
                elif (df.iloc[j]['label'] == 1):
                    label_1 = label_1 + 1
    
    suspicious_user_intensity = round(((label_1)/(label_1 + label_0)), 2)            
    print("suspicious_user_intensity: ", suspicious_user_intensity)
    df_graph.loc[i,'label'] = suspicious_user_intensity

0
suspicious_user_intensity:  0.0
1
suspicious_user_intensity:  0.11
2
suspicious_user_intensity:  0.0
3
suspicious_user_intensity:  0.0
4
suspicious_user_intensity:  0.0
5
suspicious_user_intensity:  0.0
6
suspicious_user_intensity:  0.0
7
suspicious_user_intensity:  0.0
8
suspicious_user_intensity:  0.0
9
suspicious_user_intensity:  0.85
10
suspicious_user_intensity:  0.0
11
suspicious_user_intensity:  0.31
12
suspicious_user_intensity:  0.0
13
suspicious_user_intensity:  0.0
14
suspicious_user_intensity:  0.0
15
suspicious_user_intensity:  0.0
16
suspicious_user_intensity:  0.0
17
suspicious_user_intensity:  0.27
18
suspicious_user_intensity:  0.0
19
suspicious_user_intensity:  0.23
20
suspicious_user_intensity:  0.0
21
suspicious_user_intensity:  0.0
22
suspicious_user_intensity:  0.74
23
suspicious_user_intensity:  0.0
24
suspicious_user_intensity:  0.0
25
suspicious_user_intensity:  0.0
26
suspicious_user_intensity:  0.43
27
suspicious_user_intensity:  0.49
28
suspicious_user_int

suspicious_user_intensity:  0.0
231
suspicious_user_intensity:  0.0
232
suspicious_user_intensity:  0.0
233
suspicious_user_intensity:  0.0
234
suspicious_user_intensity:  0.0
235
suspicious_user_intensity:  0.43
236
suspicious_user_intensity:  0.0
237
suspicious_user_intensity:  0.16
238
suspicious_user_intensity:  0.0
239
suspicious_user_intensity:  0.0
240
suspicious_user_intensity:  0.0
241
suspicious_user_intensity:  0.0
242
suspicious_user_intensity:  0.0
243
suspicious_user_intensity:  0.0
244
suspicious_user_intensity:  0.0
245
suspicious_user_intensity:  0.21
246
suspicious_user_intensity:  0.0
247
suspicious_user_intensity:  0.0
248
suspicious_user_intensity:  0.0
249
suspicious_user_intensity:  0.0
250
suspicious_user_intensity:  0.0
251
suspicious_user_intensity:  0.0
252
suspicious_user_intensity:  0.0
253
suspicious_user_intensity:  0.0
254
suspicious_user_intensity:  0.0
255
suspicious_user_intensity:  0.0
256
suspicious_user_intensity:  0.0
257
suspicious_user_intensity

suspicious_user_intensity:  1.0
458
suspicious_user_intensity:  1.0
459
suspicious_user_intensity:  1.0
460
suspicious_user_intensity:  1.0
461
suspicious_user_intensity:  1.0
462
suspicious_user_intensity:  1.0
463
suspicious_user_intensity:  1.0
464
suspicious_user_intensity:  1.0
465
suspicious_user_intensity:  1.0
466
suspicious_user_intensity:  1.0
467
suspicious_user_intensity:  1.0
468
suspicious_user_intensity:  1.0
469
suspicious_user_intensity:  1.0
470
suspicious_user_intensity:  1.0
471
suspicious_user_intensity:  1.0
472
suspicious_user_intensity:  1.0
473
suspicious_user_intensity:  1.0
474
suspicious_user_intensity:  1.0
475
suspicious_user_intensity:  1.0
476
suspicious_user_intensity:  1.0
477
suspicious_user_intensity:  1.0
478
suspicious_user_intensity:  1.0
479
suspicious_user_intensity:  1.0
480
suspicious_user_intensity:  1.0
481
suspicious_user_intensity:  1.0
482
suspicious_user_intensity:  1.0
483
suspicious_user_intensity:  1.0
484
suspicious_user_intensity:  

In [49]:
df_graph.iloc[553]

user_id            1.22196e+08                                     
text               jesu fuck christ americajesu fuck christ america
favorites_count    2888                                            
verified           1                                               
followers_count    358                                             
label              1                                               
Name: 553, dtype: object

In [52]:
len(list_of_initiators)

496

In [51]:
#converting labels to user suspicious score (for reply_ids)
k = -1
for i in range (len(list_of_initiators), len(df_graph)):
    k = k + 1
    print(i)
    label_0 = 0
    label_1 = 0
    for j in range(0, len(df)):
            if (list_of_replies[k] == df.iloc[j]['reply_user_id']):
                if (df.iloc[j]['reply_label'] == 0):
                    label_0 = label_0 + 1
                elif (df.iloc[j]['reply_label'] == 1):
                    label_1 = label_1 + 1
    
    suspicious_user_intensity = round(((label_1)/(label_1 + label_0)), 2)            
    print("suspicious_user_intensity: ", suspicious_user_intensity)
    df_graph.loc[i,'label'] = suspicious_user_intensity

496
suspicious_user_intensity:  1.0
497
suspicious_user_intensity:  1.0
498
suspicious_user_intensity:  0.91
499
suspicious_user_intensity:  1.0
500
suspicious_user_intensity:  1.0
501
suspicious_user_intensity:  1.0
502
suspicious_user_intensity:  0.67
503
suspicious_user_intensity:  1.0
504
suspicious_user_intensity:  1.0
505
suspicious_user_intensity:  1.0
506
suspicious_user_intensity:  1.0
507
suspicious_user_intensity:  0.29
508
suspicious_user_intensity:  0.5
509
suspicious_user_intensity:  0.0
510
suspicious_user_intensity:  1.0
511
suspicious_user_intensity:  0.5
512
suspicious_user_intensity:  1.0
513
suspicious_user_intensity:  1.0
514
suspicious_user_intensity:  0.5
515
suspicious_user_intensity:  1.0
516
suspicious_user_intensity:  1.0
517
suspicious_user_intensity:  0.33
518
suspicious_user_intensity:  1.0
519
suspicious_user_intensity:  1.0
520
suspicious_user_intensity:  0.0
521
suspicious_user_intensity:  0.14
522
suspicious_user_intensity:  1.0
523
suspicious_user_int

suspicious_user_intensity:  1.0
724
suspicious_user_intensity:  0.0
725
suspicious_user_intensity:  1.0
726
suspicious_user_intensity:  0.8
727
suspicious_user_intensity:  1.0
728
suspicious_user_intensity:  1.0
729
suspicious_user_intensity:  1.0
730
suspicious_user_intensity:  1.0
731
suspicious_user_intensity:  1.0
732
suspicious_user_intensity:  0.0
733
suspicious_user_intensity:  0.0
734
suspicious_user_intensity:  1.0
735
suspicious_user_intensity:  1.0
736
suspicious_user_intensity:  1.0
737
suspicious_user_intensity:  1.0
738
suspicious_user_intensity:  1.0
739
suspicious_user_intensity:  1.0
740
suspicious_user_intensity:  1.0
741
suspicious_user_intensity:  0.5
742
suspicious_user_intensity:  1.0
743
suspicious_user_intensity:  1.0
744
suspicious_user_intensity:  0.5
745
suspicious_user_intensity:  1.0
746
suspicious_user_intensity:  0.0
747
suspicious_user_intensity:  1.0
748
suspicious_user_intensity:  1.0
749
suspicious_user_intensity:  1.0
750
suspicious_user_intensity:  

suspicious_user_intensity:  1.0
951
suspicious_user_intensity:  0.0
952
suspicious_user_intensity:  1.0
953
suspicious_user_intensity:  0.0
954
suspicious_user_intensity:  1.0
955
suspicious_user_intensity:  0.0
956
suspicious_user_intensity:  1.0
957
suspicious_user_intensity:  1.0
958
suspicious_user_intensity:  1.0
959
suspicious_user_intensity:  1.0
960
suspicious_user_intensity:  0.0
961
suspicious_user_intensity:  0.67
962
suspicious_user_intensity:  0.75
963
suspicious_user_intensity:  1.0
964
suspicious_user_intensity:  1.0
965
suspicious_user_intensity:  1.0
966
suspicious_user_intensity:  1.0
967
suspicious_user_intensity:  1.0
968
suspicious_user_intensity:  1.0
969
suspicious_user_intensity:  1.0
970
suspicious_user_intensity:  1.0
971
suspicious_user_intensity:  1.0
972
suspicious_user_intensity:  1.0
973
suspicious_user_intensity:  0.5
974
suspicious_user_intensity:  1.0
975
suspicious_user_intensity:  0.5
976
suspicious_user_intensity:  1.0
977
suspicious_user_intensity:

suspicious_user_intensity:  0.0
1174
suspicious_user_intensity:  1.0
1175
suspicious_user_intensity:  0.0
1176
suspicious_user_intensity:  1.0
1177
suspicious_user_intensity:  1.0
1178
suspicious_user_intensity:  1.0
1179
suspicious_user_intensity:  1.0
1180
suspicious_user_intensity:  1.0
1181
suspicious_user_intensity:  0.0
1182
suspicious_user_intensity:  1.0
1183
suspicious_user_intensity:  0.0
1184
suspicious_user_intensity:  1.0
1185
suspicious_user_intensity:  1.0
1186
suspicious_user_intensity:  0.0
1187
suspicious_user_intensity:  1.0
1188
suspicious_user_intensity:  1.0
1189
suspicious_user_intensity:  1.0
1190
suspicious_user_intensity:  1.0
1191
suspicious_user_intensity:  0.83
1192
suspicious_user_intensity:  1.0
1193
suspicious_user_intensity:  1.0
1194
suspicious_user_intensity:  1.0
1195
suspicious_user_intensity:  1.0
1196
suspicious_user_intensity:  0.8
1197
suspicious_user_intensity:  1.0
1198
suspicious_user_intensity:  0.71
1199
suspicious_user_intensity:  1.0
1200

suspicious_user_intensity:  1.0
1396
suspicious_user_intensity:  1.0
1397
suspicious_user_intensity:  1.0
1398
suspicious_user_intensity:  1.0
1399
suspicious_user_intensity:  1.0
1400
suspicious_user_intensity:  1.0
1401
suspicious_user_intensity:  1.0
1402
suspicious_user_intensity:  1.0
1403
suspicious_user_intensity:  0.0
1404
suspicious_user_intensity:  1.0
1405
suspicious_user_intensity:  1.0
1406
suspicious_user_intensity:  0.5
1407
suspicious_user_intensity:  1.0
1408
suspicious_user_intensity:  0.0
1409
suspicious_user_intensity:  1.0
1410
suspicious_user_intensity:  1.0
1411
suspicious_user_intensity:  0.0
1412
suspicious_user_intensity:  1.0
1413
suspicious_user_intensity:  0.0
1414
suspicious_user_intensity:  1.0
1415
suspicious_user_intensity:  1.0
1416
suspicious_user_intensity:  1.0
1417
suspicious_user_intensity:  1.0
1418
suspicious_user_intensity:  1.0
1419
suspicious_user_intensity:  1.0
1420
suspicious_user_intensity:  1.0
1421
suspicious_user_intensity:  1.0
1422
s

...

suspicious_user_intensity:  1.0
10483
suspicious_user_intensity:  0.0
10484
suspicious_user_intensity:  0.0
10485
suspicious_user_intensity:  0.0
10486
suspicious_user_intensity:  0.0
10487
suspicious_user_intensity:  1.0
10488
suspicious_user_intensity:  0.0
10489
suspicious_user_intensity:  0.0
10490
suspicious_user_intensity:  0.0
10491
suspicious_user_intensity:  0.0
10492
suspicious_user_intensity:  0.0
10493
suspicious_user_intensity:  0.0
10494
suspicious_user_intensity:  0.0
10495
suspicious_user_intensity:  0.0
10496
suspicious_user_intensity:  0.0
10497
suspicious_user_intensity:  0.0
10498
suspicious_user_intensity:  0.0
10499
suspicious_user_intensity:  0.0
10500
suspicious_user_intensity:  0.0
10501
suspicious_user_intensity:  0.0
10502
suspicious_user_intensity:  1.0
10503
suspicious_user_intensity:  1.0
10504
suspicious_user_intensity:  1.0
10505
suspicious_user_intensity:  1.0
10506
suspicious_user_intensity:  1.0
10507
suspicious_user_intensity:  1.0
10508
suspicious_u

suspicious_user_intensity:  0.0
10699
suspicious_user_intensity:  0.0
10700
suspicious_user_intensity:  0.0
10701
suspicious_user_intensity:  0.0
10702
suspicious_user_intensity:  0.0
10703
suspicious_user_intensity:  1.0
10704
suspicious_user_intensity:  0.0
10705
suspicious_user_intensity:  0.0
10706
suspicious_user_intensity:  0.0
10707
suspicious_user_intensity:  0.0
10708
suspicious_user_intensity:  0.0
10709
suspicious_user_intensity:  0.0
10710
suspicious_user_intensity:  0.0
10711
suspicious_user_intensity:  0.0
10712
suspicious_user_intensity:  0.0
10713
suspicious_user_intensity:  0.0
10714
suspicious_user_intensity:  0.0
10715
suspicious_user_intensity:  0.0
10716
suspicious_user_intensity:  0.0
10717
suspicious_user_intensity:  0.0
10718
suspicious_user_intensity:  0.0
10719
suspicious_user_intensity:  0.0
10720
suspicious_user_intensity:  1.0
10721
suspicious_user_intensity:  0.0
10722
suspicious_user_intensity:  0.0
10723
suspicious_user_intensity:  0.0
10724
suspicious_u

suspicious_user_intensity:  0.0
10915
suspicious_user_intensity:  1.0
10916
suspicious_user_intensity:  0.0
10917
suspicious_user_intensity:  0.0
10918
suspicious_user_intensity:  0.0
10919
suspicious_user_intensity:  0.0
10920
suspicious_user_intensity:  0.0
10921
suspicious_user_intensity:  0.0
10922
suspicious_user_intensity:  0.0
10923
suspicious_user_intensity:  0.0
10924
suspicious_user_intensity:  0.0
10925
suspicious_user_intensity:  0.0
10926
suspicious_user_intensity:  0.0
10927
suspicious_user_intensity:  0.0
10928
suspicious_user_intensity:  1.0
10929
suspicious_user_intensity:  0.0
10930
suspicious_user_intensity:  0.0
10931
suspicious_user_intensity:  0.0
10932
suspicious_user_intensity:  0.0
10933
suspicious_user_intensity:  0.0
10934
suspicious_user_intensity:  0.0
10935
suspicious_user_intensity:  0.0
10936
suspicious_user_intensity:  0.0
10937
suspicious_user_intensity:  0.0
10938
suspicious_user_intensity:  0.0
10939
suspicious_user_intensity:  0.0
10940
suspicious_u

suspicious_user_intensity:  0.0
11131
suspicious_user_intensity:  0.0
11132
suspicious_user_intensity:  0.0
11133
suspicious_user_intensity:  0.0
11134
suspicious_user_intensity:  0.0
11135
suspicious_user_intensity:  0.0
11136
suspicious_user_intensity:  0.0
11137
suspicious_user_intensity:  0.0
11138
suspicious_user_intensity:  1.0
11139
suspicious_user_intensity:  0.0
11140
suspicious_user_intensity:  0.0
11141
suspicious_user_intensity:  0.0
11142
suspicious_user_intensity:  0.0
11143
suspicious_user_intensity:  0.0
11144
suspicious_user_intensity:  0.0
11145
suspicious_user_intensity:  0.0
11146
suspicious_user_intensity:  0.0
11147
suspicious_user_intensity:  0.0
11148
suspicious_user_intensity:  0.0
11149
suspicious_user_intensity:  0.0
11150
suspicious_user_intensity:  0.0
11151
suspicious_user_intensity:  0.0
11152
suspicious_user_intensity:  0.0
11153
suspicious_user_intensity:  0.0
11154
suspicious_user_intensity:  0.0
11155
suspicious_user_intensity:  0.0
11156
suspicious_u

IndexError: index 10738 is out of bounds for axis 0 with size 10738

In [53]:
df_graph.shape

(28235, 6)

In [54]:
df_graph['label'].unique()

array([0.  , 0.11, 0.85, 0.31, 0.27, 0.23, 0.74, 0.43, 0.49, 0.38, 0.7 ,
       0.05, 0.09, 0.52, 0.56, 0.55, 0.32, 0.59, 0.68, 0.25, 0.2 , 0.28,
       0.29, 0.93, 0.69, 0.33, 0.73, 0.26, 0.75, 0.4 , 0.16, 0.21, 0.86,
       0.79, 0.54, 0.44, 0.78, 0.71, 0.5 , 0.76, 0.48, 0.51, 0.88, 0.58,
       0.89, 0.62, 0.8 , 0.57, 0.35, 0.22, 1.  , 0.91, 0.67, 0.14, 0.83,
       0.82, 0.6 , 0.18, 0.19, 0.12, 0.17, 0.94, 0.72, 0.45, 0.36])

In [55]:
print(k)

10738


In [56]:
len(list_of_replies)

10738

In [57]:
len(list_of_initiators)

496

In [58]:
len(df_graph)

28235

In [79]:
df_graph.iloc[11234]

user_id            2.7749e+08                                                              
text               they need chang some thing #fergusonthey need chang some thing #ferguson
favorites_count    3439                                                                    
verified           1                                                                       
followers_count    3282                                                                    
label              0                                                                       
Name: 11234, dtype: object

In [80]:
reduce_df = df_graph[:11234]

In [81]:
reduce_df.shape

(11234, 6)

## not to be executed

In [82]:
df_graph.to_csv('graph-datav4.csv')

In [83]:
reduce_df.to_csv('graph-datav4.csv')

In [11]:
df_graph = pd.read_csv('graph-datav4.csv')

In [84]:
df_graph['user_id'].nunique()

10794

In [85]:
reduce_df['user_id'].nunique()

10793

In [86]:
df_graph.describe()

,user_id,favorites_count,verified,followers_count,label
count,2.823500e+04,28235.000000,28235.000000,2.823500e+04,28235.000000
mean,2.307857e+08,1503.225607,0.397875,8.349708e+03,0.250056
std,5.715640e+08,6654.006094,0.489468,1.809650e+05,0.424766
min,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000
25%,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000
50%,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000
75%,1.045726e+08,341.000000,1.000000,3.010000e+02,0.500000
max,2.746442e+09,306610.000000,1.000000,1.405974e+07,1.000000


In [87]:
count_0 = 0
count_1 = 0
for i in range(0, len(reduce_df)):
    if (reduce_df.iloc[i]['label'] == 0):
        count_0 = count_0 + 1
    elif (reduce_df.iloc[i]['label'] == 1):
        count_1 = count_1 + 1
print("0: ", count_0)
print("1: ", count_1)
percent_0 = (count_0) / (len(reduce_df) ) * 100
print(percent_0)

percent_1 = (count_1) / (len(reduce_df) ) * 100
print(percent_1)

0:  3763
1:  6582
33.49652839594089
58.589994659070676


In [92]:
reduce_df['label'].unique()

array([0.  , 0.14, 0.57, 0.26, 0.58, 0.89, 0.39, 0.45, 0.42, 0.59, 0.23,
       0.64, 0.47, 0.76, 0.32, 0.33, 0.75, 0.36, 0.53, 0.6 , 0.8 , 0.61,
       0.65, 0.51, 0.15, 0.24, 0.86, 0.79, 0.11, 0.19, 0.41, 0.2 , 0.85,
       0.77, 0.66, 0.56, 0.82, 0.87, 0.55, 0.5 , 0.37, 0.88, 0.52, 0.49,
       0.68, 0.95, 0.81, 0.83, 0.44, 0.4 , 1.  , 0.67, 0.78, 0.7 , 0.22,
       0.43, 0.25, 0.94, 0.29, 0.38, 0.62, 0.48, 0.06, 0.17, 0.71])

In [89]:
reduce_df.shape

(11234, 6)

In [88]:
df_graph.shape

(28235, 6)

# Manual Word2Vec

## Word Embeddings using gensim

In [90]:
import gensim

In [30]:
model = gensim.models.Word2Vec(sentences = text, size = 100, window = 2, workers = 3, min_count = 1)

In [31]:
words = list(model.wv.vocab)
print('Vocab size', len(words))

Vocab size 22604


In [32]:
words

['charli',
 'hebdo',
 'becam',
 'well',
 'known',
 'publish',
 'muham',
 'cartoon',
 'year',
 'dead',
 'shoot',
 'today',
 'guess',
 'thi',
 'consid',
 'terror',
 'right',
 'lone',
 'wolf',
 'would',
 'mention',
 'befor',
 'know',
 'fact',
 'islamphobiaatitsbest',
 'perp',
 'identifi',
 'french',
 'satir',
 'magazin',
 'mayb',
 'becaus',
 'shout',
 'prophet',
 'aveng',
 'thu',
 'make',
 'relev',
 'stori',
 'sourc',
 'heard',
 'report',
 'sever',
 'black',
 'cagoul',
 'wrote',
 'pierr',
 'cossett',
 'broadcast',
 'journalist',
 'europ',
 'new',
 'hope',
 'help',
 'mate',
 'http',
 'ubrbsr',
 'quit',
 'sure',
 'call',
 'remain',
 'open',
 'busi',
 'pray',
 'soul',
 'murder',
 'recoveri',
 'injur',
 'come',
 'friendli',
 'gestur',
 'clearli',
 'expect',
 'islamaphob',
 'anyway',
 'provid',
 'defens',
 'argument',
 'arrest',
 'blasphemi',
 'incit',
 'violenc',
 'muslim',
 'best',
 'scare',
 'truth',
 'moslem',
 'peac',
 'islam',
 'bloodi',
 'hand',
 'anoth',
 'rampag',
 'becom',
 'systemat

## Most Similar Words

In [33]:
model.wv.most_similar('terror')

[('strongli', 0.9937201738357544),
 ('barbar', 0.992141604423523),
 ('rgmnfzda', 0.9919722080230713),
 ('condem', 0.9910004734992981),
 ('react', 0.9908586144447327),
 ('respons', 0.9898620843887329),
 ('wake', 0.9891852736473083),
 ('cowardli', 0.9890744686126709),
 ('messag', 0.9885111451148987),
 ('say', 0.98848956823349)]

## Save vectors

In [34]:
file = 'twitter_charlie_manual_word2vec.txt'

In [35]:
model.wv.save_word2vec_format(file,binary = False)

C:\ProgramData\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [36]:
import os
emb = {}
f = open(os.path.join('', 'twitter_graph_charlie_manual_word2vec.txt'), encoding = 'utf-8')
for line in f:
    values = line.split()
    word = values[0]
    value = np.asarray(values[1:])
    emb[word] = value
f.close()

In [37]:
emb['well']

array(['0.47268343', '0.21874283', '0.43794572', '0.97547567',
       '0.27692533', '-0.15025121', '-1.4180529', '-0.37245232',
       '0.07424749', '-0.22060248', '-1.0551224', '-1.2201867',
       '-0.04113183', '-0.45489827', '-0.1602611', '0.5162439',
       '0.12073903', '-0.7184269', '-0.40073958', '-0.18289664',
       '-0.64492095', '0.3486478', '0.44549343', '-0.11237968',
       '0.0019082659', '-0.78128976', '-0.011435941', '-0.058834936',
       '-1.3285239', '0.48364866', '-0.28005126', '0.45026678',
       '0.4606849', '0.05232132', '-0.30942258', '-0.41936913',
       '-0.3984525', '-0.0695967', '-0.7217789', '0.66084385',
       '-0.4576076', '-0.594535', '-0.28899294', '0.36177835',
       '0.7462315', '-0.25699258', '0.099692196', '-1.035578',
       '-0.18189456', '-0.3405407', '0.84291095', '0.7929444',
       '-0.6748584', '0.80534136', '0.2111648', '0.35548708',
       '-0.74710625', '-0.75889766', '0.4196848', '0.1825899',
       '0.6099316', '-0.24377652', '-0.6

In [54]:
token = Tokenizer()
token.fit_on_texts(text)
seq = token.texts_to_sequences(text)
index = token.word_index
print('unique no of tokens ', index)
padding = pad_sequences(seq, maxlen = max_length)
rumour = df_graph['label'].values
print('Shape of tweet ',padding.shape)
print('Shape of rumour ', rumour.shape)

unique no of tokens  {'http': 1, 'thi': 2, 'charliehebdo': 3, 'muslim': 4, 'kill': 5, 'islam': 6, 'attack': 7, 'peopl': 8, 'pari': 9, 'religion': 10, 'terrorist': 11, 'like': 12, 'jesuischarli': 13, 'cartoon': 14, 'charli': 15, 'franc': 16, 'know': 17, 'right': 18, 'freedom': 19, 'think': 20, 'hebdo': 21, 'speech': 22, 'hostag': 23, 'french': 24, 'world': 25, 'make': 26, 'polic': 27, 'would': 28, 'terror': 29, 'free': 30, 'onli': 31, 'thank': 32, 'want': 33, 'need': 34, 'time': 35, 'good': 36, 'murder': 37, 'well': 38, 'support': 39, 'say': 40, 'break': 41, 'becaus': 42, 'take': 43, 'live': 44, 'tweet': 45, 'stop': 46, 'call': 47, 'fuck': 48, 'said': 49, 'even': 50, 'condemn': 51, 'also': 52, 'show': 53, 'christian': 54, 'veri': 55, 'peac': 56, 'point': 57, 'satir': 58, 'read': 59, 'agre': 60, 'never': 61, 'mean': 62, 'media': 63, 'mani': 64, 'thing': 65, 'realli': 66, 'shoot': 67, 'pleas': 68, 'believ': 69, 'look': 70, 'must': 71, 'victim': 72, 'love': 73, 'suspect': 74, 'hate': 75, '

In [55]:
padding[13]

array([   0,    0,    0, ..., 1136, 2602,  625])

In [56]:
rumour

array([0., 0., 0., ..., 1., 1., 1.])

In [57]:
n = len(index) + 1
embed_mat = np.zeros((n, 100))
for s,values in index.items():
    print(s,values)
    if values>n:
        continue
    embed_vec = emb.get(s)
    if embed_vec is not None:
        embed_mat[values] = embed_vec
print(n)

http 1
thi 2
charliehebdo 3
muslim 4
kill 5
islam 6
attack 7
peopl 8
pari 9
religion 10
terrorist 11
like 12
jesuischarli 13
cartoon 14
charli 15
franc 16
know 17
right 18
freedom 19
think 20
hebdo 21
speech 22
hostag 23
french 24
world 25
make 26
polic 27
would 28
terror 29
free 30
onli 31
thank 32
want 33
need 34
time 35
good 36
murder 37
well 38
support 39
say 40
break 41
becaus 42
take 43
live 44
tweet 45
stop 46
call 47
fuck 48
said 49
even 50
condemn 51
also 52
show 53
christian 54
veri 55
peac 56
point 57
satir 58
read 59
agre 60
never 61
mean 62
media 63
mani 64
thing 65
realli 66
shoot 67
pleas 68
believ 69
look 70
must 71
victim 72
love 73
suspect 74
hate 75
come 76
prophet 77
cartoonist 78
doe 79
dead 80
noth 81
sure 82
name 83
today 84
countri 85
publish 86
everi 87
much 88
human 89
give 90
happen 91
report 92
hope 93
tell 94
still 95
respect 96
stand 97
wrong 98
violenc 99
true 100
news 101
express 102
word 103
work 104
differ 105
follow 106
jew 107
offend 108
understand 1

doesnt 1356
ad 1357
oblig 1358
pixel 1359
version 1360
duti 1361
fascism 1362
senseless 1363
principl 1364
highest 1365
fast 1366
prosecutor 1367
lesson 1368
unfold 1369
niet 1370
arabia 1371
understood 1372
former 1373
lawyer 1374
object 1375
attitud 1376
isnt 1377
scream 1378
guid 1379
fill 1380
cameron 1381
super 1382
piti 1383
specul 1384
antisemit 1385
engag 1386
contribut 1387
divid 1388
got 1389
ta 1390
ourselv 1391
random 1392
conclus 1393
scholar 1394
wolinski 1395
launcher 1396
rock 1397
comparison 1398
sentenc 1399
terrifi 1400
soldier 1401
tear 1402
republish 1403
mad 1404
workplac 1405
horror 1406
viva 1407
tool 1408
aussi 1409
search 1410
boy 1411
suicid 1412
divis 1413
impli 1414
union 1415
rip 1416
sleep 1417
taliban 1418
barbarian 1419
plant 1420
approv 1421
wise 1422
sydney 1423
organis 1424
dick 1425
steal 1426
corner 1427
particularli 1428
fanatic 1429
squar 1430
francestrong 1431
fame 1432
gitmo 1433
hijack 1434
communist 1435
credit 1436
jazeera 1437
trocadero 143

delud 2058
eat 2059
pervert 2060
fortun 2061
nigel 2062
galleri 2063
initi 2064
wwii 2065
convict 2066
bash 2067
regular 2068
poster 2069
arent 2070
youth 2071
road 2072
eventu 2073
await 2074
affair 2075
mislead 2076
wan 2077
hors 2078
meanwhil 2079
fit 2080
frame 2081
pair 2082
embarrass 2083
pointless 2084
deflect 2085
ala 2086
musulman 2087
disturb 2088
region 2089
sexual 2090
misunderstand 2091
obscen 2092
welfar 2093
favor 2094
assimil 2095
daughter 2096
wave 2097
independ 2098
nigerian 2099
timelin 2100
reader 2101
demonis 2102
permit 2103
art 2104
secularist 2105
naacpbomb 2106
pero 2107
grew 2108
music 2109
sing 2110
bloodbath 2111
essenti 2112
mankind 2113
benghazi 2114
manner 2115
mankoff 2116
dialect 2117
sorrow 2118
tini 2119
roman 2120
shia 2121
master 2122
qayyahmoynihan 2123
endors 2124
instanc 2125
egyptian 2126
flog 2127
cow 2128
properti 2129
fuckyouterroristcoward 2130
perp 2131
shove 2132
sarcast 2133
algerian 2134
soft 2135
sit 2136
jenesuispascharli 2137
buy 2138

estim 2761
reliabl 2762
digniti 2763
suffici 2764
arsehol 2765
cope 2766
badli 2767
personn 2768
votr 2769
gang 2770
lectur 2771
untru 2772
complex 2773
sai 2774
humbl 2775
suddenli 2776
memo 2777
guillotin 2778
season 2779
beef 2780
est 2781
shape 2782
grammar 2783
msnbc 2784
practis 2785
conquest 2786
sono 2787
photograph 2788
colon 2789
thinker 2790
thick 2791
brigad 2792
mlkul 2793
oxxv 2794
wcar 2795
jackaddawkin 2796
threw 2797
humili 2798
joostniemol 2799
fled 2800
amnesti 2801
stuck 2802
deliv 2803
fuckin 2804
began 2805
hideou 2806
legisl 2807
dilut 2808
nake 2809
warm 2810
element 2811
dunno 2812
contrast 2813
jackson 2814
ele 2815
heheh 2816
chri 2817
volcano 2818
exclus 2819
preserv 2820
preciou 2821
rear 2822
gracia 2823
blatant 2824
imageri 2825
famil 2826
angel 2827
golden 2828
spill 2829
project 2830
exclud 2831
imprison 2832
somewhat 2833
match 2834
vision 2835
newsroom 2836
pavement 2837
firearm 2838
plaster 2839
pander 2840
jesui 2841
dame 2842
maker 2843
resolut 284

handsupdontshoot 4101
holster 4102
carnag 4103
draconian 4104
distast 4105
wimp 4106
tail 4107
bollywood 4108
sectarian 4109
spout 4110
foundat 4111
ceux 4112
discret 4113
kowtow 4114
moisi 4115
insight 4116
fraternit 4117
wale 4118
becuas 4119
comeback 4120
shaw 4121
represent 4122
libtard 4123
charm 4124
neighbour 4125
conclud 4126
disapprov 4127
yike 4128
moor 4129
freepress 4130
tackl 4131
elder 4132
island 4133
ikoyi 4134
symptom 4135
airtim 4136
debt 4137
dispatch 4138
clariti 4139
mensch 4140
faux 4141
complaint 4142
csa 4143
craze 4144
compound 4145
drawmohammedday 4146
simmer 4147
bcuz 4148
bundi 4149
pentagon 4150
oklahoma 4151
funer 4152
crush 4153
indonesia 4154
venom 4155
yawn 4156
primarili 4157
horrid 4158
armour 4159
vest 4160
biard 4161
raul 4162
castro 4163
uobdtgyj 4164
peaceful 4165
regul 4166
summari 4167
ash 4168
lunch 4169
reassur 4170
unrel 4171
monopoli 4172
bd 4173
april 4174
juif 4175
barrier 4176
summer 4177
temer 4178
juvenil 4179
pajglm 4180
dessin 4181
or

...

slime 19625
varmint 19626
koombuyyah 19627
efnk 19628
wpiasbxa 19629
fzscfrt 19630
ldvuhlf 19631
lawmak 19632
cond 19633
gzuscb 19634
jbvityth 19635
pwnedit 19636
gxlhlp 19637
gtyzutl 19638
gobsmack 19639
ykul 19640
articul 19641
jesuissiraj 19642
unsuccess 19643
senseperson 19644
nobigd 19645
quenel 19646
nujopp 19647
exkdefuobp 19648
ramif 19649
bikcvxgo 19650
newsweek 19651
gloo 19652
ktjp 19653
mazb 19654
huffpo 19655
ohhzhotiwd 19656
zhpiirqmva 19657
qyfqh 19658
shedload 19659
dyidk 19660
banana 19661
ktog 19662
gdqpkmdhttp 19663
tytokmpbvr 19664
gxffiqub 19665
xkbuv 19666
gpft 19667
npmrokl 19668
flutter 19669
uneas 19670
unrepent 19671
cppkxaagla 19672
gcqcuhq 19673
unpunish 19674
ialntnitx 19675
datelin 19676
malici 19677
sheffield 19678
blunkett 19679
pickpocket 19680
bedvqtynp 19681
scous 19682
ginger 19683
vicar 19684
dibley 19685
bollo 19686
profoundli 19687
historyrepeatsitself 19688
holedig 19689
dura 19690
shoukd 19691
shortag 19692
yrpq 19693
zkkk 19694
brbwdq 19695
lcp

feliz 20332
saludo 20333
adema 20334
solidario 20335
nombr 20336
expres 20337
ggeuab 20338
emlnyvbguk 20339
tolyl 20340
tryna 20341
redflag 20342
bmmp 20343
profag 20344
hadnt 20345
subscipt 20346
notebook 20347
rofpcgv 20348
enrol 20349
finjoxnpf 20350
evkydl 20351
jahddk 20352
iuwj 20353
numhcd 20354
wjqlfomnq 20355
nibuohjlz 20356
lbkhxhff 20357
decompos 20358
kkfncdwxmc 20359
ggrehlkwi 20360
autocraci 20361
uyoir 20362
goofyaward 20363
lilgoebbelsaward 20364
hcmcuyqa 20365
tkbbvpwl 20366
thluwbhvq 20367
pngujtekbr 20368
lnxj 20369
ndbpz 20370
cqefwv 20371
jamuqdsd 20372
google 20373
zeaka 20374
qkksk 20375
xcept 20376
aiyarterrorlogicgood 20377
rsxgmgvyi 20378
pirhqag 20379
wbndkopeut 20380
yvrdxtyio 20381
dkcugca 20382
devour 20383
cillatert 20384
whet 20385
feenz 20386
dtto 20387
effin 20388
hmrbpzw 20389
victima 20390
sido 20391
asesinada 20392
kjrevl 20393
worsert 20394
yugiylbjf 20395
gxshrhlqfn 20396
encajiplhf 20397
paxd 20398
ycfdrgf 20399
pylcieauhn 20400
belarusian 20401


neeeeeeeexxxxxxt 21043
aywrf 21044
eyhxfrpl 21045
lpgvj 21046
reduct 21047
uhwno 21048
toso 21049
wsle 21050
equin 21051
cocoa 21052
lhiyf 21053
belndi 21054
jrbhbxkzmi 21055
mhpue 21056
pinx 21057
xmggb 21058
corp 21059
jlaop 21060
babpk 21061
ntkgqpd 21062
vcudiib 21063
mprha 21064
vfnirmnnlm 21065
qtmizest 21066
iuhmlbsa 21067
ffqg 21068
dongectomi 21069
conspirest 21070
noaflt 21071
hkjzxwevzd 21072
qaxtqmjvz 21073
maeuqnda 21074
aqqbtxnddr 21075
hjdcdfggdi 21076
jyox 21077
dnrt 21078
crank 21079
hunk 21080
nostalgia 21081
ytqdj 21082
sgwchvgg 21083
nent 21084
souvent 21085
comport 21086
tueur 21087
moosad 21088
measli 21089
thee 21090
queri 21091
ngrvl 21092
zbdf 21093
plenm 21094
snack 21095
arrr 21096
dood 21097
cartoonsa 21098
riiiihgt 21099
xuju 21100
bvfpk 21101
contraataqu 21102
qyvndr 21103
lngysofvu 21104
ink 21105
fxaux 21106
uomljd 21107
zhetrvucm 21108
rdpagtk 21109
kcyrnmb 21110
vgjii 21111
bcdxnti 21112
usza 21113
ohmygod 21114
bhpcevwepc 21115
cwonksamzj 21116
insyaa

inew 21735
inquiet 21736
choc 21737
succinct 21738
qcfbpdnh 21739
ttyf 21740
thierri 21741
mesan 21742
solicit 21743
xjnbphgj 21744
fbnz 21745
txxfi 21746
microchip 21747
aipmauehpn 21748
rfid 21749
scareomet 21750
pushmor 21751
vorratsdatenspeicherung 21752
spyag 21753
wylhdrza 21754
zwcoqlgi 21755
mgxgyw 21756
apenforpari 21757
royceysanytimechalleng 21758
wguguipxtzprinc 21759
ocshi 21760
livr 21761
ambassad 21762
londr 21763
fusillad 21764
jkvpbsd 21765
porker 21766
abovert 21767
hlhi 21768
duke 21769
responsibl 21770
ragbr 21771
jhjlb 21772
citizenradio 21773
differntli 21774
egg 21775
ador 21776
shutter 21777
hahahahhahhahahahahahhaha 21778
fjkzbrnhk 21779
umov 21780
hfan 21781
xjqhxufaal 21782
zcontoiflj 21783
nfvsakli 21784
profond 21785
pensez 21786
wkapzeefrp 21787
cranki 21788
uufzfawdw 21789
qrxx 21790
respectforallreligion 21791
gebr 21792
contrelesterrorist 21793
atentadoparisep 21794
jesuischarliearv 21795
xckqu 21796
amrabet 21797
moroccan 21798
pubesc 21799
provis 2180

upris 22453
chowder 22454
thisisajokeandthereforemeanttobetaken 22455
suchsopleasedon 22456
tmurderm 22457
kalt 22458
fmsd 22459
lyeszgpal 22460
wkeplw 22461
sooooft 22462
mpbpljkt 22463
reconnaiss 22464
gta 22465
mogelijk 22466
berust 22467
toeval 22468
airforc 22469
vqofpwlv 22470
arme 22471
forcesfield 22472
jkcyl 22473
flir 22474
liam 22475
neeson 22476
craignez 22477
treilli 22478
compri 22479
emploi 22480
rent 22481
pourtant 22482
riterait 22483
sourceon 22484
arabisch 22485
bronnen 22486
waarop 22487
zich 22488
baseert 22489
unverifi 22490
spraken 22491
mosoel 22492
tijden 22493
vrijdagsgeb 22494
aangekondigd 22495
volgt 22496
ongetwijfeld 22497
boubak 22498
hakim 22499
voorlopig 22500
afwachten 22501
soort 22502
dingen 22503
vaag 22504
ieder 22505
reden 22506
verkondigen 22507
vind 22508
bizar 22509
hebben 22510
kloppen 22511
rondhoren 22512
contacten 22513
terecht 22514
bedenk 22515
ontken 22516
binnen 22517
verschillend 22518
leden 22519
vallen 22520
lucht 22521
natuurlijk 22

In [58]:
embed_mat.shape

(22605, 100)

In [59]:
len(index)

22604

In [60]:
sorted_words = sorted(index)
print(sorted_words)

['aaaaaaand', 'aaaaaand', 'aaaaah', 'aabbasi', 'aabijw', 'aadmi', 'aadmyjw', 'aadsavmgh', 'aalo', 'aameen', 'aamer', 'aamir', 'aamirkhan', 'aand', 'aandacht', 'aangekondigd', 'aanhalen', 'aankh', 'aanleid', 'aanslag', 'aanslagen', 'aanval', 'aaosbod', 'aapk', 'aar', 'aargh', 'aaron', 'aashish', 'aatankwad', 'aavd', 'aaxbv', 'abadon', 'aban', 'abandon', 'abaraham', 'abashislamis', 'abat', 'abatido', 'abatt', 'abattoir', 'abattr', 'abavn', 'abba', 'abboutfacegenladi', 'abdest', 'abdo', 'abdomen', 'abdul', 'aber', 'aberr', 'abet', 'abhi', 'abhor', 'abhorr', 'abid', 'abiderswow', 'abigail', 'abil', 'abject', 'abjo', 'abkqgtokct', 'abl', 'abla', 'ableist', 'abnsolutli', 'abod', 'abohr', 'abolish', 'abolishreligion', 'abomin', 'abondonn', 'abonn', 'aboout', 'abort', 'aborto', 'abot', 'abound', 'aboutit', 'abov', 'abovert', 'abqo', 'abraham', 'abrahm', 'abramje', 'abrazo', 'abroad', 'abrog', 'abruti', 'absenc', 'absent', 'absolu', 'absolut', 'absoluta', 'absolutli', 'absorb', 'absorto', 'abst

In [61]:
index.keys()

dict_keys(['http', 'thi', 'charliehebdo', 'muslim', 'kill', 'islam', 'attack', 'peopl', 'pari', 'religion', 'terrorist', 'like', 'jesuischarli', 'cartoon', 'charli', 'franc', 'know', 'right', 'freedom', 'think', 'hebdo', 'speech', 'hostag', 'french', 'world', 'make', 'polic', 'would', 'terror', 'free', 'onli', 'thank', 'want', 'need', 'time', 'good', 'murder', 'well', 'support', 'say', 'break', 'becaus', 'take', 'live', 'tweet', 'stop', 'call', 'fuck', 'said', 'even', 'condemn', 'also', 'show', 'christian', 'veri', 'peac', 'point', 'satir', 'read', 'agre', 'never', 'mean', 'media', 'mani', 'thing', 'realli', 'shoot', 'pleas', 'believ', 'look', 'must', 'victim', 'love', 'suspect', 'hate', 'come', 'prophet', 'cartoonist', 'doe', 'dead', 'noth', 'sure', 'name', 'today', 'countri', 'publish', 'everi', 'much', 'human', 'give', 'happen', 'report', 'hope', 'tell', 'still', 'respect', 'stand', 'wrong', 'violenc', 'true', 'news', 'express', 'word', 'work', 'differ', 'follow', 'jew', 'offend', '

## Map words with their vectors

In [62]:
features = {}

In [63]:
d = np.arange(0, len(padding))

In [64]:
data = np.zeros((0, len(df_graph)))

In [65]:
feat_df = pd.DataFrame(index = d, data = data, columns = sorted_words)
print(feat_df)

      aaaaaaand  aaaaaand  aaaaah  aabbasi  aabijw  aadmi  aadmyjw  aadsavmgh  \
0     0.0        0.0       0.0     0.0      0.0     0.0    0.0      0.0         
1     0.0        0.0       0.0     0.0      0.0     0.0    0.0      0.0         
2     0.0        0.0       0.0     0.0      0.0     0.0    0.0      0.0         
3     0.0        0.0       0.0     0.0      0.0     0.0    0.0      0.0         
4     0.0        0.0       0.0     0.0      0.0     0.0    0.0      0.0         
5     0.0        0.0       0.0     0.0      0.0     0.0    0.0      0.0         
6     0.0        0.0       0.0     0.0      0.0     0.0    0.0      0.0         
7     0.0        0.0       0.0     0.0      0.0     0.0    0.0      0.0         
8     0.0        0.0       0.0     0.0      0.0     0.0    0.0      0.0         
9     0.0        0.0       0.0     0.0      0.0     0.0    0.0      0.0         
10    0.0        0.0       0.0     0.0      0.0     0.0    0.0      0.0         
11    0.0        0.0       0

In [66]:
feat_df.shape

(1006, 22604)

In [67]:
padding[4]

array([  0,   0,   0, ..., 106, 125, 644])

In [68]:
embed_mat[2].mean()

-0.07120083937179998

In [69]:
feat_df.iloc[1][1] == 0.0

True

In [70]:
#combining features with average
for row in range(0, len(padding)):
    print(row)
    for number in padding[row]:
        for key,value in index.items():
            if number == value:
                mean = embed_mat[number].mean()
                if (feat_df.iloc[row][key] == 0.0):
                    feat_df.iloc[row][key] = mean
                else:
                    feat_df.iloc[row][key] = feat_df.iloc[row][key] + mean

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [72]:
feat_df.to_csv('features of word2vec embeddings.csv')

In [77]:
feat_df=feat_df.fillna(0)

In [80]:
feat_df.shape

(1006, 22604)

## Combining Features into 1 dataframe

In [85]:
df_graph.drop('clean_text', axis = 1, inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [98]:
df_graph.head()

,src_followers_count,src_favorites_count,src_verified_user,label
0,41591,0,1,0.0
1,139847,216,1,0.0
2,22568,58,1,0.0
3,2327,612,1,0.0
4,8134,344,1,0.0


In [99]:
X_new = pd.concat([feat_df, df_graph], axis=1)

In [100]:
X_new.shape

(1006, 22608)

X_new.to_csv('charlie_combined_word2vec_features_with_original_features.csv')

# Graph

In [ ]:
G=nx.Graph()

for i in range(0, len(df)):
    G.add_edge(df.iloc[i]['src_user_id'], df.iloc[i]['reply_user_id'], weight = df.iloc[i]['weight'])

nx.degree(G)

In [ ]:
print(nx.average_clustering(G))
nx.density(G)

In [ ]:
nx.is_connected(G)

**Adjacency matrix**

In [ ]:
A = nx.adjacency_matrix(G)

In [ ]:
A.shape